In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import datetime

In [4]:
mongo_client = MongoClient('54.207.44.6', 27017)
mongo_db = mongo_client['inatel']
mongo_collection = mongo_db['in242']

In [5]:
dados = list(mongo_collection.find())
ed = pd.DataFrame(dados)

ServerSelectionTimeoutError: 54.207.44.6:27017: timed out

In [110]:
ed.to_csv('consolidado.csv')

In [6]:
df = pd.read_csv('consolidado.csv')

In [7]:
df.tail()

,Unnamed: 0,_id,temperatura,data_coleta
133907,133907,5dbe2643136d09329b96f6e5,24.843006,2019-11-02 21:58:43.105
133908,133908,5dbe2644136d09329b96f6e6,17.642706,2019-11-02 21:58:44.107
133909,133909,5dbe2645136d09329b96f6e7,23.430738,2019-11-02 21:58:45.108
133910,133910,5dbe2646136d09329b96f6e8,19.663868,2019-11-02 21:58:46.110
133911,133911,5dbe2647136d09329b96f6e9,27.568199,2019-11-02 21:58:47.111


In [8]:
df.count()

Unnamed: 0     133912
_id            133912
temperatura    133912
data_coleta    133912
dtype: int64

In [114]:
df = df.drop(columns=['_id'])

In [9]:
df = df.drop(columns=['Unnamed: 0'])

In [10]:
df.tail()

,_id,temperatura,data_coleta
133907,5dbe2643136d09329b96f6e5,24.843006,2019-11-02 21:58:43.105
133908,5dbe2644136d09329b96f6e6,17.642706,2019-11-02 21:58:44.107
133909,5dbe2645136d09329b96f6e7,23.430738,2019-11-02 21:58:45.108
133910,5dbe2646136d09329b96f6e8,19.663868,2019-11-02 21:58:46.110
133911,5dbe2647136d09329b96f6e9,27.568199,2019-11-02 21:58:47.111


In [11]:
def check_temperatura(temperatura):
    if temperatura < 21:
        return 'Frio'
    if temperatura < 26:
        return 'Normal'
    return 'Quente'

In [12]:
df['sensacao'] = df['temperatura'].apply(check_temperatura)

In [13]:
df.groupby(['sensacao']).count()

,_id,temperatura,data_coleta
sensacao,,,
Frio,41329,41329,41329
Normal,51174,51174,51174
Quente,41409,41409,41409


In [14]:
df.groupby(['sensacao'])['temperatura'].mean()

sensacao
Frio      18.997327
Normal    23.495629
Quente    28.006566
Name: temperatura, dtype: float64

In [16]:
df.describe()

,temperatura
count,133912.000000
mean,23.502223
std,3.762209
min,17.000020
25%,20.235575
50%,23.500988
75%,26.775257
max,29.999897
